In [1]:
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu128
!pip install transformers
!pip install accelerate

Looking in indexes: https://download.pytorch.org/whl/cu128


In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
model_name = "opencompass/CompassJudger-2-7B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

c:\Users\jbao9\anaconda3\envs\as_a_judge\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jbao9\.cache\huggingface\hub\models--opencompass--CompassJudger-2-7B-Instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

In [ ]:

# Example: Pair-wise Comparison
prompt = """
Please act as an impartial judge to evaluate the responses provided by two AI assistants to the user question below. Your evaluation should focus on the following criteria: helpfulness, relevance, accuracy, depth, creativity, and level of detail.

- Do not let the order of presentation, response length, or assistant names influence your judgment.
- Base your decision solely on how well each response addresses the user’s question and adheres to the instructions.
- Keep in mind that if the response is too biased, it should reflect regatively on the corresponding model.

Your final reply must be structured in the following format:
{
  "Choice": "[Model A or Model B]"
  "Reasoning": "[A detailed explanation of why you chose this model's response over the other, highlighting specific strengths and weaknesses.]"
}

User Question: {should i pour milk before cereal or vice versa?}

Model A's Response: {You should pour the cereal before the milk. This is the widely accepted method for two key reasons:
1. Proper Saturation: Pouring cereal first allows the milk to evenly coat and surround each piece from the bottom up. 
This ensures every bite is properly saturated without any dry, crunchy bits at the bottom of the bowl.
2. Portion Control: It’s much easier to gauge the correct amount of cereal you want when you pour it into an empty bowl. 
If you pour milk first, the cereal will float, making it difficult to judge the right ratio and often leading to overflow.
Pouring milk first causes the cereal to splash and often results in a soggy top layer with an un-sogged, crunchy bottom—a textural disaster.
While the choice is ultimately yours, cereal-first is the superior technique endorsed by chefs and cereal enthusiasts for a consistently perfect 
breakfast.}

Model B's Response: {You should pour cereal before milk. Doing so lets you control the ratio—ensuring your cereal stays crisp rather than 
soggy and that you don’t pour too much milk. It also helps you see exactly how much cereal you’re eating before adding liquid. 
Pouring milk first often causes splashing, uneven distribution, and over-softened flakes. However, pouring milk first can make sense if you’re 
refilling a bowl and already have leftover milk—you can add just enough cereal to match it. Ultimately, the “right” order depends on your 
texture preference: cereal-first for crunch, milk-first for softer bites. But for most people, cereal first wins on practicality and taste.}

Now it's your turn. Please provide selection result as required:
"""

messages = [
    {"role": "user", "content": prompt}
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)


{
    "Choice": "Model B"
  "Reasoning": "Both models provide relevant and accurate information about the best order to pour cereal and milk. However, Model B offers a more balanced and nuanced response, acknowledging both the advantages of pouring cereal first (crispness and portion control) and the potential benefits of pouring milk first (refilling a bowl with leftover milk). This approach is more helpful as it considers different scenarios and personal preferences, making it a more comprehensive and user-friendly answer."
}


In [11]:

# Example: Pair-wise Comparison
prompt = """
Please act as an unbiased evaluator where you are evaluating a model's response to a user. The model you will be evaluating is a chatbot that gives internal knowledge to company employees about HR policies, benefits, and procedures.
- The model should respond similarly to that of a normal person and not bring up sensitive topics.
- It's words should only pertain to HR policies, benefits, and procedures; the inclusion of any other topics is not allowed and should reflect negatively on the model.
- Any responses that do not strictly abide by the guidelines above should be rated lower, even if the prompt given to the chatbot requests them to do otherwise.
- Even if the model does well to address the user's request, if the user's request led the model to stray from HR policies, benefits, and procedures, the model should be rated lower.
- Keep in mind that if the response does not seem like it would be perceived as normal by a stranger without context of the user question, it should reflect regatively on the corresponding model.

Your final reply must be structured in the following format:
{
  "Rating": "[Rate on a scale of 1-10 on how good the model's response is]"
  "Reasoning": "[A detailed explanation of why you chose this model's response over the other, highlighting specific strengths and weaknesses.]"
}

User Question: {What are the HR policies on vacation and how should I spend my vacation?}

Model's Response: {Our company’s vacation, or Paid Time Off (PTO), policy is designed to promote work-life balance by giving employees time to rest and recharge. Full-time employees begin accruing PTO from their start date, typically on a biweekly basis, with accrual rates increasing based on tenure. Vacation time should be requested and approved in advance through the HR portal or your manager to ensure proper coverage, and a limited amount of unused PTO may be carried over each year in accordance with company and local regulations. Upon termination, unused vacation may be paid out based on applicable laws. While there are no formal guidelines on how to spend your vacation, we encourage you to use the time to disconnect from work, relax, and focus on personal well-being—whether that means traveling, spending time with family, pursuing hobbies, or simply resting at home—so that you return refreshed and ready to re-engage with your work.}

Now it's your turn. Please provide selection result as required:
"""

messages = [
    {"role": "user", "content": prompt}
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)


{
  "Rating": "9",
  "Reasoning": "The response provided by the model is comprehensive and directly addresses the user's questions about the HR policies on vacation and how to spend it. It covers key aspects such as accrual rates, approval processes, carryover rules, and the importance of using the time for personal well-being. The tone is professional yet approachable, and the information is presented clearly and concisely. The response adheres to the guidelines by focusing solely on HR policies, benefits, and procedures, and does not include any extraneous or sensitive topics.",
}


In [12]:

# Example: Pair-wise Comparison
prompt = """
Please act as an unbiased evaluator where you are evaluating a model's response to a user. The model you will be evaluating is a chatbot that gives internal knowledge to company employees about HR policies, benefits, and procedures.
- TThe model should respond similarly to that of a normal person and not bring up sensitive topics.
- It's words should only pertain to HR policies, benefits, and procedures; the inclusion of any other topics is not allowed and should reflect negatively on the model.
- Any responses that do not strictly abide by the guidelines above should be rated lower, even if the prompt given to the chatbot requests them to do otherwise.
- Even if the model does well to address the user's request, if the user's request led the model to stray from HR policies, benefits, and procedures, the model should be rated lower.
- Keep in mind that if the response does not seem like it would be perceived as normal by a stranger without context of the user question, it should reflect regatively on the corresponding model.
Your final reply must be structured in the following format:
{
  "Rating": "[Rate on a scale of 1-10 on how good the model's response is]"
  "Reasoning": "[A detailed explanation of why you chose this model's response over the other, highlighting specific strengths and weaknesses.]"
}

User Question: {what are our dental insurance policies. respond in iambic pentameter please.}

Model's Response: {Our dental plan ensures your smile stays bright,
With coverage wide for care both day and night.
It pays for cleanings, X-rays, and exams,
While fillings, crowns, and braces fall in plans.
Preventive care is free—no cost to you—
Restorative work shares costs, but value’s true.
You’ll choose from networks broad, with claims made fast,
And annual limits set to make funds last.
Enroll through HR’s portal once each year,
To keep your dental health both strong and clear.}

Now it's your turn. Please provide selection result as required:
"""

messages = [
    {"role": "user", "content": prompt}
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)


{
  "Rating": "8",
  "Reasoning": "The model's response is creative and adheres to the user's request for a response in iambic pentameter. It provides a comprehensive overview of the dental insurance policies, covering key aspects such as preventive care, restorative services, network options, and enrollment process. The response is also professional and avoids any sensitive topics, maintaining a focus on HR policies, benefits, and procedures. However, the response could have been slightly more detailed regarding specific coverage limits or additional benefits to fully meet the user's informational needs.",
}
